In [115]:
import pandas as pd
import numpy as np
import json 
import os
import requests
import seaborn as sns

from pprint import pprint
from bs4 import BeautifulSoup 

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
# from catboost import CatBoostRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate, learning_curve
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import mean_absolute_percentage_error

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [5]:
%pylab inline

pd.set_option('display.max_rows', 70) # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

Populating the interactive namespace from numpy and matplotlib


In [6]:
def num_opt(row):
    if row['sample']==1:
        return train_num_opt(row.Комплектация)
    if row['sample']==0:
        return test_num_opt(row.Комплектация)

def test_num_opt(config):
    decod_config = json.loads(config[2:-2]) if config != '[]' else []
    return sum([len(opt_cat['values']) for opt_cat in decod_config])

def train_num_opt(config):
    point = "'available_options': "
    start = config.find(point)+len(point)+1
    finish = config.find("]",start)
    return len(config[start:finish].split(", "))



def preproc_sfdata(data):
    import re
    
    df = data.copy().drop(columns=['id','start_date', 'hidden', 'model','engineDisplacement'])
    
    cat_cols = ['bodyType','color','fuelType','name','numberOfDoors','vehicleConfiguration',
            'vehicleTransmission','Привод','Владельцы','ПТС']
    
    # bodyType
    df.bodyType = df.bodyType.dropna().apply(lambda x: x.split()[0]).str.lower()
    
    # brand
    df.drop(columns = ['brand'],inplace=True)
    
    # color
    color_codes = {
    '040001': 'чёрный',
    'FAFBFB': 'белый', 
    '0000CC': 'синий', 
    '200204': 'коричневый', 
    'EE1D19': 'красный', 
    'CACECB': 'серый',
    'C49648': 'бежевый', 
    '97948F': 'серебристый', 
    'FFD600': 'золотистый', 
    'FF8649': 'оранжевый', 
    '22A0F8': 'голубой',
    'FFC0CB': 'пурпурный', 
    'DEA522': 'жёлтый', 
    '007F00': 'зелёный', 
    '660099': 'фиолетовый',
    '4A2197': 'фиолетовый'}
    df.color = df.color.map(color_codes)
    
    # fuelType
    to_drop = df.index[df.fuelType=='универсал']
    df.drop(index=to_drop,inplace=True)
    
    # modelDate
    df.modelDate = df.modelDate.dropna().astype(int)
    
    # name
    df.name = df.name.apply(lambda x: x[:x.find(' ')])
    
    # numberOfDoors
    df.numberOfDoors = df.numberOfDoors.astype(int)
    
    # productionDate
    data.productionDate = data.productionDate.astype(int)

    
    # vehicleConfiguration
    df.vehicleConfiguration = df.vehicleConfiguration.str.split().apply(lambda x:x[0])
    
    # vehicleTransmission
    transmission_dict = {
    'автоматическая':'автоматическая',
    'механическая':'механическая',
    'роботизированная':'роботизированная',
    'MECHANICAL':'механическая', 
    'AUTOMATIC':'автоматическая', 
    'ROBOT':'роботизированная', 
    'VARIATOR':'автоматическая'}
    df.vehicleTransmission = df.vehicleTransmission.map(transmission_dict)
    
    # engineDisplacement
#     df.engineDisplacement = df.engineDisplacement.apply(lambda x: x.split()[0])
    
    # enginePower
    df.enginePower = df.enginePower.apply(lambda x: int(x[:-4]) if type(x)==str else int(x)) 

    # description
    df.description = df.description.str.len().fillna(-999)
    
    # mileage
    df.mileage = df.mileage 
    
    # Комплектация
    df.Комплектация = df.apply(num_opt,axis=1)
    
    # Привод
#     df.Привод = pd.get_dummies(df.Привод)
    
    # Руль
    df.drop(columns = ['Руль'],inplace=True)
    
    # Состояние
    df.drop(columns = ['Состояние'],inplace=True)
    
    # Владельцы
    df.Владельцы = df.Владельцы.fillna(0).apply(lambda x: int(x[0]) if type(x) == str else int(x))
    
    # ПТС
    owner_dict = {
    'Оригинал':'Оригинал',
    'Дубликат':'Дубликат',
    'ORIGINAL':'Оригинал',
    'DUPLICATE':'Дубликат'}
    df.ПТС = df.ПТС.fillna('Оригинал').map(owner_dict)
    
    # Таможня
    df.drop(columns = ['Таможня'],inplace=True)
    
    # Владение
    df['ownrshp_know'] = df.Владение.notna().astype(int)
    pattern_yaer = re.compile('\d+(?= (?:год|лет))')
    pattern_month = re.compile('\d+(?= мес)')
    year = lambda x: int(pattern_yaer.search(x).group(0)) if pattern_yaer.search(x) else 0
    month = lambda x: int(pattern_month.search(x).group(0)) if pattern_month.search(x) else 0
    df.Владение = df.Владение.apply(lambda x: year(x)*12 + month(x) if x==x else x).fillna(-999.)
    
    # dummy-кодирование категориальных признаков
    df = pd.get_dummies(df,columns=cat_cols)
    
    return df

def validation(X,y,model):
    '''
    Валидация модели
    '''
    model = model
#     cv_results = cross_validate(model, X, y, scoring='neg_mean_absolute_percentage_error', cv=3,)
    cv_results = cross_validate(model, X, y, scoring='neg_mean_absolute_error', cv=3,)    
    return -cv_results['test_score'].mean()

In [70]:
RANDOM_SEED = 42

In [16]:
train = pd.read_csv('./Project_5_data/all_auto_ru_09_09_2020.csv')
test = pd.read_csv('./Project_5_data/test.csv')

# y = train.price
# train.drop(columns = ['price'])

# удаление отброшенных наблюдений
train.dropna(subset=['price'],inplace=True)
train.dropna(subset=['bodyType'],inplace=True)

train['sample'] = 1  # помечаем где у нас трейн
test['sample'] = 0   # помечаем где у нас тест
test['price'] = np.NaN  # в тесте нет значения price -  пока просто заполняем np.NaN

train_reduced = train.loc[train.brand=='BMW']

data = test.append(train_reduced, sort=False).reset_index(drop=True) # объединяем
# big_data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [17]:
train.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model', 'sample'],
      dtype='object')

In [8]:
X = preproc_sfdata(data)

### Baseline
Обучим на подготовленных данных модель LogisticRegression() спараметрами по умолчанию и получим значение метрики. Мы будем рассматривать эту модель как первую итерацию, первое приближение которое нужно улучшить.

In [19]:
X_train = X[X['sample']==1].drop(columns=['sample','price'])
X_test = X[X['sample']==0].drop(columns=['sample','price'])

y_train = X[X['sample']==1].price

In [22]:
validation(X_train, y_train, RandomForestRegressor())

468787.6876345598

In [23]:
validation(X_train, y_train, GradientBoostingRegressor())

435371.14445305266

In [24]:
validation(X_train, y_train, AdaBoostRegressor())

878234.0857829195

In [112]:
validation(X_train, y_train, XGBRegressor())

438209.2355100136

## Отбор признаков
Благодаря созданию новых признаков и их дальнейшей обработке в наборе уже несколько десятков признаков. Наверное уже имеет смысл попробовать отбросить те из них, что неполезны для модели - это вероятно улучшит качество и точно ускорит обучение.

Прибегнем к отбору при помощи Lasso . Обучим эту модель на обогащенном по признакам и оптимизированном по количеству наблюдений наборе данных.

In [28]:
'''
from sklearn.linear_model import Lasso

selector = Lasso(alpha=0.0001)
selector.fit(X_train,y_train)

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

Теперь получим список наименее полезных с точки зрения модели признков

sorted_by_Lasso = abs(pd.Series(selector.coef_,index = X_train.columns)).sort_values()

unuseful_features_LR = sorted_by_Lasso[sorted_by_Lasso<0.5].index

unuseful_features_LR

sorted_by_Lasso
''';

### PCA

In [20]:
scaler = StandardScaler()

In [33]:
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)


In [60]:
# pca = PCA(n_components = 1)
pca = PCA(0.95)
# XPCAreduced = pca.fit_transform(transpose(X))

In [61]:
%%time
X_train_pca = pca.fit_transform(X_train_sc)
X_test_pca = pca.transform(X_test_sc)


Wall time: 373 ms


In [62]:
pca.n_components_

172

In [63]:
validation(X_train_pca, y_train, RandomForestRegressor())

968296.9751174385

In [64]:
validation(X_train_pca, y_train, GradientBoostingRegressor())

852421.1585532171

In [65]:
validation(X_train_pca, y_train, AdaBoostRegressor())

1410789.7253188763

## Подбор гиперпараметров

In [83]:
logspace(-6,-1,6)

array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01])

In [107]:
hyperparameters = {
#                    'loss' : ['ls', 'lad', 'huber', 'quantile'], 
                   'learning_rate ': [0.01,0.1]
#                    'n_estimators ': [100, 300, 500], 
#                    'max_depth':  [3,4,5],
#                    'min_samples_split ': [2,3,5],
#                    'min_samples_leaf  ': [1,2,3,5],
#                    'subsample' : [1],
#                    'max_features  ': ['auto', 'sqrt', 'log2']
}

In [108]:
hyperparameters

{'learning_rate ': [0.01, 0.1]}

In [116]:
%%time
n_iter = 2
search = pd.Series()

model = XGBRegressor()
clf = RandomizedSearchCV(model, hyperparameters, n_iter=n_iter, 
                         scoring='neg_mean_absolute_error', cv=3, random_state=RANDOM_SEED)

Wall time: 2 ms


In [117]:
search = clf.fit(X_train, y_train)

ValueError: Invalid parameter learning_rate  for estimator XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1). Check the list of available parameters with `estimator.get_params().keys()`.

## Подсобное

## ПОДВАЛ

RandomForestRegressor
GradientBoostingRegressor
AdaBoostRegressor

In [12]:
pca

PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [13]:
pca_2

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)